<a href="https://colab.research.google.com/github/Spectrewolf8/Ascii-Nova-Ascii-video-renderer-and-player/blob/main/shaham_com_1_fruits_classifier_v2_backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Experimental**

In [ ]:
!pip install tensorflow opencv-python matplotlib huggingface_hub

In [ ]:
# !pip install -r /content/drive/MyDrive/#shaham_com_1_fruits_classifier_v2-backend_storage/requirements.txt

In [ ]:
# Cell 1: Imports and Constants
import tensorflow as tf
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import cv2

In [ ]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5
FRUIT_TYPES = ['Apple', 'Banana', 'Guava', 'Lime', 'Orange', 'Pomegranate']
QUALITY_TYPES = ['Good', 'Bad']

In [ ]:
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

#logging into Hugging Face
!huggingface-cli login --token $hf_token

In [ ]:
from huggingface_hub import hf_hub_download
import tensorflow as tf

# Define your model repository and file name
repo_name = "spectrewolf8/MobileNetv3_fruit_quality_classifier"
filename = "MobileNetv3_fruit_quality_classifier.h5"  # Update this to the actual model file name

# Define the directory where you want to save the downloaded model
download_dir = "/kaggle/working/spectrewolf8/cache/"

# Download the model file from Hugging Face Hub to the specified directory
local_model_path = hf_hub_download(repo_id=repo_name, filename=filename, cache_dir=download_dir)

# !rm - rf /kaggle/working/spectrewolf8/cache/models--spectrewolf8--aerial-image-road-segmentation-xp

# Load the model with custom objects
model = tf.keras.models.load_model(local_model_path)


# Check the model summary
model.summary()

In [ ]:
def generate_gradcam(model, image, layer_name='Conv1', target_size=(IMG_SIZE, IMG_SIZE)):
    # Build the grad model - flatten the outputs list
    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[model.get_layer(layer_name).output] + model.outputs  # Flatten the output list
    )

    # Add a gradient tape to monitor the computation of gradients
    with tf.GradientTape() as tape:
        # Forward pass: Get the layer's output and the model's output
        conv_outputs, fruit_type_output, quality_output = grad_model(image, training=False)

        # Combine losses for both outputs
        loss = tf.reduce_mean(fruit_type_output) + tf.reduce_mean(quality_output)

    # Calculate the gradients of the combined loss w.r.t. conv layer outputs
    grads = tape.gradient(loss, conv_outputs)

    # Pool gradients across the spatial dimensions (average them out)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Process the convolution outputs
    conv_outputs = conv_outputs[0]

    # Generate the Grad-CAM heatmap
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)

    # Resize the heatmap to the target size (200x200)
    heatmap = cv2.resize(heatmap.numpy(), target_size, interpolation=cv2.INTER_LINEAR)

    return heatmap

In [ ]:

# def explain_prediction(model, image_path):
#     # Load and preprocess the image
#     img = tf.keras.preprocessing.image.load_img(
#         image_path, target_size=(IMG_SIZE, IMG_SIZE)
#     )
#     img_array = tf.keras.preprocessing.image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0) / 255.0

#     # Make predictions
#     fruit_pred, quality_pred = model.predict(img_array)

#     # Generate Grad-CAM heatmap
#     heatmap = generate_gradcam(model, img_array)

#     # Plot the original image, heatmap, and heatmap only
#     plt.figure(figsize=(15, 5))

#     plt.subplot(1, 3, 1)
#     plt.imshow(img)
#     plt.title('Original Image')

#     plt.subplot(1, 3, 2)
#     plt.imshow(img)
#     plt.imshow(heatmap, alpha=0.6, cmap='jet')
#     plt.title('GradCAM Heatmap')

#     plt.subplot(1, 3, 3)
#     plt.imshow(heatmap, cmap='jet')
#     plt.title('Heatmap Only')

#     plt.show()

#     # Get predictions and their confidence
#     predicted_fruit = FRUIT_TYPES[np.argmax(fruit_pred)]
#     fruit_confidence = np.max(fruit_pred)

#     predicted_quality = QUALITY_TYPES[np.argmax(quality_pred)]
#     quality_confidence = np.max(quality_pred)

#     # Create explanatory text
#     explanation = (
#         f"Prediction for the image:\n"
#         f"- Predicted Fruit: {predicted_fruit} (Confidence: {fruit_confidence:.2f})\n"
#         f"- Predicted Quality: {predicted_quality} (Confidence: {quality_confidence:.2f})\n\n"
#         f"Insights:\n"
#         f"- The model is highly confident in the fruit prediction if the confidence is above 0.7.\n"
#         f"- Quality prediction confidence helps in understanding the ripeness and edibility of the fruit. \n"
#         f"- The Grad-CAM heatmap indicates the areas of the image that were most influential in making the predictions."
#     )

#     return {
#         'fruit_type': predicted_fruit,
#         'fruit_confidence': fruit_confidence,
#         'quality': predicted_quality,
#         'quality_confidence': quality_confidence,
#         'explanation': explanation
#     }


# def explain_prediction(model, image_path):
#     # Load and preprocess the image
#     img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
#     img_array = tf.keras.preprocessing.image.img_to_array(img)
#     img_array = np.expand_dims(img_array, axis=0) / 255.0

#     # Make predictions
#     fruit_pred, quality_pred = model.predict(img_array)

#     # Generate Grad-CAM heatmap
#     heatmap = generate_gradcam(model, img_array)  # Assuming you have the `generate_gradcam` function

#     # Save the original image, heatmap, and Grad-CAM overlay images
#     result_dir = 'static/results'
#     os.makedirs(result_dir, exist_ok=True)

#     original_image_path = os.path.join(result_dir, 'original_image.jpg')
#     heatmap_path = os.path.join(result_dir, 'heatmap.jpg')
#     overlay_path = os.path.join(result_dir, 'overlay_image.jpg')

#     # Save the original image
#     plt.imsave(original_image_path, img)

#     # Save the heatmap-only image
#     plt.imsave(heatmap_path, heatmap, cmap='jet')

#     # Save the overlay (original image + heatmap)
#     plt.figure(figsize=(15, 5))
#     plt.imshow(img)
#     plt.imshow(heatmap, alpha=0.6, cmap='jet')
#     plt.axis('off')
#     plt.savefig(overlay_path)
#     plt.close()

#     # Get predictions and their confidence
#     predicted_fruit = FRUIT_TYPES[np.argmax(fruit_pred)]
#     fruit_confidence = np.max(fruit_pred)

#     predicted_quality = QUALITY_TYPES[np.argmax(quality_pred)]
#     quality_confidence = np.max(quality_pred)

#     # Create explanatory text
#     explanation = (
#         f"Prediction for the image:\n"
#         f"- Predicted Fruit: {predicted_fruit} (Confidence: {fruit_confidence:.2f})\n"
#         f"- Predicted Quality: {predicted_quality} (Confidence: {quality_confidence:.2f})\n\n"
#         f"Insights:\n"
#         f"- The model is highly confident in the fruit prediction if the confidence is above 0.7.\n"
#         f"- Quality prediction confidence helps in understanding the ripeness and edibility of the fruit. \n"
#         f"- The Grad-CAM heatmap indicates the areas of the image that were most influential in making the predictions."
#     )

#     return {
#         'fruit_type': predicted_fruit,
#         'fruit_confidence': fruit_confidence,
#         'quality': predicted_quality,
#         'quality_confidence': quality_confidence,
#         'explanation': explanation,
#         'original_image_path': original_image_path,
#         'heatmap_path': heatmap_path,
#         'overlay_path': overlay_path
#     }

In [ ]:
# Example prediction
test_image_path = '/content/sample_images/IMG202007281550021.jpg'  # Replace with actual test image path
predictions = explain_prediction(model, test_image_path)

print("Predictions:")
print(f"Fruit Type: {predictions['fruit_type']} (Confidence: {predictions['fruit_confidence'] * 100:.2f}%)")
print(f"Quality: {predictions['quality']} (Confidence: {predictions['quality_confidence'] * 100:.2f}%)")

# **Compute Backend** - actual

In [ ]:
!pip install --ignore-installed tensorflow==2.17.0 flask pyngrok jinja2 pymongo Flask-PyMongo
# --ignore-installed

In [ ]:
!pip install shap

In [ ]:
from huggingface_hub import hf_hub_download
import tensorflow as tf
import json
import h5py
import numpy as np
# Define your model repository and file name
repo_name = "spectrewolf8/MobileNetv3_fruit_quality_classifier"
filename = "MobileNetv3_fruit_quality_classifier.h5"  # Update this to the actual model file name

# Define the directory where you want to save the downloaded model
download_dir = "/content/drive/MyDrive/#shaham_com_1_fruits_classifier_v2-backend_storage/model/"

# Download the model file from Hugging Face Hub to the specified directory
local_model_path = hf_hub_download(repo_id=repo_name, filename=filename, cache_dir=download_dir)

def load_model_with_custom_objects():
    custom_objects = {
        'Functional': tf.keras.Model,
    }

    try:
        # First attempt: direct loading
        model = tf.keras.models.load_model(local_model_path, custom_objects=custom_objects)
        return model
    except (ValueError, TypeError) as e:
        print(f"Standard loading failed. Attempting alternative loading method. Error: {str(e)}")

        # Second attempt: manual reconstruction
        try:
            with h5py.File(local_model_path, 'r') as f:
                model_config = f.attrs.get('model_config')
                if model_config is None:
                    raise ValueError("No model configuration found in the H5 file")

                # Decode model config if it's in bytes
                if isinstance(model_config, bytes):
                    model_config = model_config.decode('utf-8')

                # Parse the config
                model_config = json.loads(model_config)

                # Create the model from config
                model = tf.keras.models.model_from_json(json.dumps(model_config), custom_objects=custom_objects)

                # Load weights manually
                weight_names = [name for name in f.keys() if 'layer' in name]
                for name in weight_names:
                    g = f[name]
                    weights = [np.array(g[wname]) for wname in g.keys()]
                    model.get_layer(name).set_weights(weights)

                return model
        except Exception as e:
            print(f"Alternative loading method failed. Error: {str(e)}")

            # Third attempt: try loading as SavedModel format
            try:
                model = tf.keras.models.load_model(local_model_path, custom_objects=custom_objects)
                return model
            except Exception as e:
                print(f"All loading attempts failed. Final error: {str(e)}")
                raise

# Usage
try:
    model = load_model_with_custom_objects()
    print("Model loaded successfully!")
except Exception as e:
    print(f"Failed to load model: {str(e)}")

model.summary()

In [ ]:
import shap
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.models import Model

# Constants
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5
FRUIT_TYPES = ['Apple', 'Banana', 'Guava', 'Lime', 'Orange', 'Pomegranate']
QUALITY_TYPES = ['Good', 'Bad']

def generate_gradcam(model, image, layer_name, class_idx=None):
    """
    Generate Grad-CAM heatmap for the specified layer and class index.

    Args:
        model: Trained TensorFlow model
        image: Preprocessed input image (normalized, expanded dims)
        layer_name: Name of the target convolutional layer
        class_idx: Index of the target class (None for multi-output models)

    Returns:
        Normalized heatmap
    """
    # Get the target layer
    grad_model = Model(
        inputs=[model.inputs],
        outputs=[model.get_layer(layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(image)
        if isinstance(predictions, list):
            # For multi-output model, sum up the outputs
            loss = sum(tf.reduce_sum(pred) for pred in predictions)
        else:
            if class_idx is None:
                loss = tf.reduce_sum(predictions)
            else:
                loss = predictions[:, class_idx]

    # Calculate gradients
    grads = tape.gradient(loss, conv_output)

    # Global average pooling
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # Weight the channels by corresponding gradients
    conv_output = conv_output[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_output), axis=-1)

    # ReLU
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)

    return heatmap.numpy()

def process_and_overlay_heatmap(image_path, heatmap, target_size=(IMG_SIZE, IMG_SIZE)):
    """
    Process the heatmap and overlay it on the original image.

    Args:
        image_path: Path to the original image
        heatmap: Generated heatmap
        target_size: Size to resize the heatmap and image

    Returns:
        original_img: Resized original image
        processed_heatmap: Processed heatmap image
        overlaid_img: Heatmap overlaid on original image
    """
    # Load and resize original image
    original_img = cv2.imread(image_path)
    original_img = cv2.resize(original_img, target_size)
    original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)

    # Process heatmap
    heatmap = cv2.resize(heatmap, target_size)
    heatmap = np.uint8(255 * heatmap)
    processed_heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    processed_heatmap = cv2.cvtColor(processed_heatmap, cv2.COLOR_BGR2RGB)

    # Overlay heatmap on original image
    overlaid_img = cv2.addWeighted(original_img, 0.7, processed_heatmap, 0.3, 0)

    return original_img, processed_heatmap, overlaid_img

def calculate_shap_values(model, image_array, background_images=None, num_samples=100):

    # Ensure we have the correct image dimensions
    if len(image_array.shape) == 3:
        image_array = np.expand_dims(image_array, axis=0)

    # Create background if not provided
    if background_images is None:
        background_images = np.zeros((1,) + image_array.shape[1:])

    # Create separate models for each output
    fruit_model = Model(inputs=model.input, outputs=model.outputs[0])
    quality_model = Model(inputs=model.input, outputs=model.outputs[1])

    # Create GradientExplainer for each output
    fruit_explainer = shap.GradientExplainer(fruit_model, background_images)
    quality_explainer = shap.GradientExplainer(quality_model, background_images)

    # Calculate SHAP values
    fruit_shap_values = fruit_explainer.shap_values(image_array, nsamples=num_samples)
    quality_shap_values = quality_explainer.shap_values(image_array, nsamples=num_samples)

    # Handle different shapes of SHAP values
    if isinstance(fruit_shap_values, list):
        fruit_shap_values = [np.array(v) for v in fruit_shap_values]
    if isinstance(quality_shap_values, list):
        quality_shap_values = [np.array(v) for v in quality_shap_values]

    return {
        'fruit_shap_values': fruit_shap_values,
        'quality_shap_values': quality_shap_values
    }

def save_shap_visualization(shap_values, image_array, save_path):
    """
    Create and save SHAP visualization for 5D SHAP values.

    Args:
        shap_values: SHAP values with shape (batch, height, width, channels, num_classes)
        image_array: Original image array
        save_path: Path to save the visualization
    """
    plt.figure(figsize=(10, 6))

    # Remove the batch dimension and sum across classes
    shap_values_reshaped = np.sum(shap_values[0], axis=-1)  # Now shape is (height, width, channels)

    # Create the visualization
    shap.image_plot([shap_values_reshaped], image_array[0], show=False)

    plt.savefig(save_path)
    plt.close()

def explain_prediction_vbackend(model, image_path, result_dir, target_layer='Conv_1'):
    """
    Generate and save explanation visualizations including SHAP values for the model's prediction.
    """
    # Existing setup code remains the same
    os.makedirs(result_dir, exist_ok=True)
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Make predictions
    fruit_pred, quality_pred = model.predict(img_array)

    # Calculate SHAP values
    shap_results = calculate_shap_values(model, img_array)

    # Add this after calculating shap_results
    print("Fruit SHAP values type:", type(shap_results['fruit_shap_values']))
    if isinstance(shap_results['fruit_shap_values'], list):
        print("Fruit SHAP values shapes:", [v.shape for v in shap_results['fruit_shap_values']])
    else:
        print("Fruit SHAP values shape:", shap_results['fruit_shap_values'].shape)

    print("Quality SHAP values type:", type(shap_results['quality_shap_values']))
    if isinstance(shap_results['quality_shap_values'], list):
        print("Quality SHAP values shapes:", [v.shape for v in shap_results['quality_shap_values']])
    else:
        print("Quality SHAP values shape:", shap_results['quality_shap_values'].shape)

    # Add debugging information
    print("Fruit SHAP values shape:", shap_results['fruit_shap_values'].shape)
    print("Quality SHAP values shape:", shap_results['quality_shap_values'].shape)

    # Generate Grad-CAM heatmap
    heatmap = generate_gradcam(model, img_array, target_layer)
    original_img, heatmap_img, overlay_img = process_and_overlay_heatmap(image_path, heatmap)

    # Save images
    file_paths = {}
    for img_type, img_data in [
        ('original', original_img),
        ('heatmap', heatmap_img),
        ('overlay', overlay_img)
    ]:
        save_path = os.path.join(result_dir, f'{img_type}_image.jpg')
        plt.imsave(save_path, img_data)
        file_paths[f'{img_type}_path'] = f'/static/results/{img_type}_image.jpg'

    try:
        # Save SHAP visualizations for each output
        fruit_shap_path = os.path.join(result_dir, 'fruit_shap.jpg')
        save_shap_visualization(shap_results['fruit_shap_values'], img_array, fruit_shap_path)
        file_paths['fruit_shap_path'] = f'/static/results/fruit_shap.jpg'

        quality_shap_path = os.path.join(result_dir, 'quality_shap.jpg')
        save_shap_visualization(shap_results['quality_shap_values'], img_array, quality_shap_path)
        file_paths['quality_shap_path'] = f'/static/results/quality_shap.jpg'
    except Exception as e:
        print(f"Error generating SHAP visualizations: {str(e)}")
        # If SHAP visualization fails, we still want to return other results

    # Rest of your function remains the same
    predicted_fruit = FRUIT_TYPES[np.argmax(fruit_pred)]
    fruit_confidence = float(np.max(fruit_pred))
    predicted_quality = QUALITY_TYPES[np.argmax(quality_pred)]
    quality_confidence = float(np.max(quality_pred))

    # Enhanced explanation with SHAP information
    explanation = (
        f"Prediction Results:\n"
        f"- Fruit: {predicted_fruit} (Confidence: {fruit_confidence:.1%})\n"
        f"- Quality: {predicted_quality} (Confidence: {quality_confidence:.1%})\n\n"
        f"Visualization Guide:\n"
        f"1. Grad-CAM Heatmap:\n"
        f"   - Highlights regions most important for the model's decision\n"
        f"   - Brighter colors (red/yellow) indicate higher importance\n"
        f"   - Some Images might not have a Grad-Cam, try changing angles and conditions in that case\n\n"
        f"2. SHAP Values:\n"
        f"   - Shows pixel-level contributions to the prediction\n"
        f"   - Red indicates positive impact, blue indicates negative impact\n"
        f"   - Separate visualizations for fruit type and quality predictions"
    )

    return {
        'fruit_type': predicted_fruit,
        'fruit_confidence': f"{fruit_confidence:.1%}",
        'quality': predicted_quality,
        'quality_confidence': f"{quality_confidence:.1%}",
        'explanation': explanation,
        **file_paths
    }

In [ ]:
import os
import threading
from flask import Flask, request, jsonify, render_template, redirect, url_for, flash, session
from flask_pymongo import PyMongo
from werkzeug.security import generate_password_hash, check_password_hash
from werkzeug.utils import secure_filename
from pyngrok import ngrok, conf
import tensorflow as tf
import numpy as np
import datetime
import getpass
from google.colab import userdata
import sys

# Set the current working directory to where your template files are located
project_dir = '/content/drive/MyDrive/#shaham_com_1_fruits_classifier_v2-backend_storage'
os.chdir(project_dir)
print(f"Current working directory: {os.getcwd()}")

# Flush prints immediately
def print_flush(*args, **kwargs):
    print(*args, **kwargs)
    sys.stdout.flush()

# Your NGROK_TOKEN should be set securely in Colab
NGROK_TOKEN = userdata.get('NGROK_TOKEN')
MONGODB_ATLAS_TOKEN = userdata.get('MONGODB_ATLAS_URI')

# Constants
IMG_SIZE = 224
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
UPLOAD_FOLDER = '/content/drive/MyDrive/#shaham_com_1_fruits_classifier_v2-backend_storage/static/uploads'
RESULT_FOLDER = '/content/drive/MyDrive/#shaham_com_1_fruits_classifier_v2-backend_storage/static/results'

# Function to create directories if they don't exist
if not os.path.exists(UPLOAD_FOLDER):
    os.makedirs(UPLOAD_FOLDER)

if not os.path.exists(RESULT_FOLDER):
    os.makedirs(RESULT_FOLDER)

app = Flask(__name__)
app.config['SECRET_KEY'] = '@123'  # Change this to a secure secret key
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['RESULT_FOLDER'] = RESULT_FOLDER
app.config['MONGO_URI'] = f'{MONGODB_ATLAS_TOKEN}/fruits_classifier_v2_users_db'  # Replace with your MongoDB Atlas connection string

# print(app.config['MONGO_URI'])

# Initialize MongoDB
mongo = PyMongo(app)

# Helper function to check allowed files
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

# Define your routes (as before)
@app.route('/')
def index():
    if 'username' in session:
        return render_template('submission.html', username=session['username'])
    return render_template('login.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        users = mongo.db.users
        existing_user = users.find_one({'username': request.form['username']})

        if existing_user is None:
            hashpass = generate_password_hash(request.form['password'])
            users.insert_one({
                'username': request.form['username'],
                'password': hashpass
            })
            session['username'] = request.form['username']
            return redirect(url_for('submission'))

        flash('Username already exists!')
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        users = mongo.db.users
        login_user = users.find_one({'username': request.form['username']})

        if login_user:
            if check_password_hash(login_user['password'], request.form['password']):
                session['username'] = request.form['username']
                return redirect(url_for('submission'))

        flash('Invalid username/password combination')
    return render_template('login.html')

@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('index'))

@app.route('/submission', methods=['GET', 'POST'])
def submission():
    if 'username' not in session:
        return redirect(url_for('login'))

    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part!')
            return redirect(request.url)

        file = request.files['file']
        if file.filename == '':
            flash('No selected file!')
            return redirect(request.url)

        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(filepath)

            # Store the filename in the session so it can be used in the results page
            session['uploaded_file_path'] = filepath

            return redirect(url_for('results'))

        flash('Invalid file type!')
        return redirect(request.url)

    return render_template('submission.html', username=session['username'])

@app.route('/results')
def results():
    if 'username' not in session:
        return redirect(url_for('login'))

    if 'uploaded_file_path' not in session:
        flash('No file uploaded!')
        return redirect(url_for('submission'))

    filepath = session['uploaded_file_path']

    # Run your model prediction
    result = explain_prediction_vbackend(model, filepath, app.config['RESULT_FOLDER'])

    # Ensure proper type conversion for confidence values
    try:
        result['fruit_confidence'] = float(result['fruit_confidence'])
        result['quality_confidence'] = float(result['quality_confidence'])
    except (ValueError, TypeError):
        # If conversion fails, ensure they're strings
        result['fruit_confidence'] = str(result['fruit_confidence'])
        result['quality_confidence'] = str(result['quality_confidence'])
        result['timestamp'] = datetime.datetime.utcnow()
    # Save the results to the MongoDB database
    mongo.db.predictions.insert_one({
        'username': session['username'],
        'filename': os.path.basename(filepath),
        'fruit_type': result['fruit_type'],
        'fruit_confidence': result['fruit_confidence'],
        'quality': result['quality'],
        'quality_confidence': result['quality_confidence'],
        'timestamp': datetime.datetime.utcnow()
    })

    return render_template('results.html', result=result)


# Function to start ngrok in a thread
def start_ngrok():
    conf.get_default().auth_token = NGROK_TOKEN
    port = 5000
    public_url = ngrok.connect(port)
    print_flush(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# Start ngrok in a new thread
threading.Thread(target=start_ngrok).start()

# Run Flask app normally (main thread)
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)


# **CLI/CMDS** - dispensable

In [ ]:
!mkdir /content/sample_images

In [ ]:
pwd_dir = '/content/drive/MyDrive/#shaham_com_1_fruits_classifier_v2-backend_storage'

In [ ]:
!pwd

In [ ]:
!mkdir '/content/static/uploads'

In [ ]:
!mv /content/sample_images/ /content/drive/MyDrive/#shaham_com_1_fruits_classifier_v2-backend_storage/

In [ ]:
!rm -rf /content/static/

In [ ]:
!cd $dir

In [ ]:
!pip freeze > '/content/drive/MyDrive/#shaham_com_1_fruits_classifier_v2-backend_storage/requirements.txt'